<h3>1. Import Necessary Libraries</h3>
<ul>
  <li><strong>folium</strong> for mapping.</li>
  <li><strong>pandas</strong> for data manipulation.</li>
  <li><strong>numpy</strong> for numerical operations.</li>
  <li><strong>re</strong> for regular expressions.</li>
</ul>

In [1]:
%pip install folium
import folium 
import pandas as pd
import numpy as np
import re

<h3>2. Load Data from CSV Files</h3>
<p>Data is loaded into pandas DataFrames from:</p>
<ul>
  <li><code>ACT_Speed_Zones.csv</code></li>
  <li><code>Traffic_speed_camera_locations.csv</code></li>
  <li><code>locations_ref.csv</code></li>
<p>these contain information about speed zones, camera locations and some reference information.</p>

In [2]:
speedzones= pd.read_csv("ACT_Speed_Zones.csv")

speedcameralocations = pd.read_csv("Traffic_speed_camera_locations.csv")

locations_ref = pd.read_csv("locations_ref.csv")

<h3>3. Processes the <code>speedzones</code> DataFrame</h3>
<p>Perform several operations to extract and clean latitude and longitude information from the <code>the_geom</code> column, which is initially in <code>MULTILINESTRING</code> format. This involves separating this geometry data into start and end coordinates (latitude and longitude) for each speed zone.</p>

In [3]:

#seperate out lat longs from speed zones

speedzones['the_geom'] = speedzones['the_geom'].replace('MULTILINESTRING \(\(', '', regex=True)
speedzones['the_geom'] = speedzones['the_geom'].replace('\)\)', '', regex=True)

speedzones['startlatlong'] = speedzones['the_geom'].replace(',.*', '', regex=True)
speedzones['endlatlong'] = speedzones['the_geom'].replace('.*,', '', regex=True)


#trim endlatlong
speedzones.endlatlong = speedzones.endlatlong.str.strip()

speedzones['startlat'] = speedzones['startlatlong'].replace('.*\s', '', regex=True)
speedzones['startlong'] = speedzones['startlatlong'].replace('\s.*', '', regex=True)

speedzones['endlat'] = speedzones['endlatlong'].replace('.*\s', '', regex=True)
speedzones['endlong'] = speedzones['endlatlong'].replace('\s.*', '', regex=True)

speedzones['startlong'] = speedzones['startlong'].apply(float)
speedzones['startlat'] = speedzones['startlat'].apply(float)

speedzones['endlong'] = speedzones['endlong'].apply(float)
speedzones['endlat'] = speedzones['endlat'].apply(float)

<h3>4. Merge DataFrames</h3>
<p>Merges <code>speedcameralocations</code> with <code>locations_ref</code> on a common 'LOCATION_CODE' to enrich camera location data with additional reference information then clean up by removing the redundant colums.</p>

In [4]:
#pull in the street names based on location codes
speedcameralocations = pd.merge(speedcameralocations, locations_ref, how='inner', left_on=['LOCATION_CODE'], right_on=['Camera_Location'])

#remove second location column 
speedcameralocations = speedcameralocations.drop('Camera_Location', axis=1)

<h3>5. Update Road Name Abbreviations</h3>
<p>Updates road name abbreviations to their full forms in the <code>speedzones</code> DataFrame for consistency and clarity in analysis.</p>

In [5]:

#add long name to road name
speedzones['ROAD_NAME'] = speedzones["ROAD_NAME"].replace('ST', 'STREET', regex=True)
speedzones['ROAD_NAME'] = speedzones["ROAD_NAME"].replace('DR', 'DRIVE', regex=True)
speedzones['ROAD_NAME'] = speedzones["ROAD_NAME"].replace('CR', 'CRESCENT', regex=True)
speedzones['ROAD_NAME'] = speedzones["ROAD_NAME"].replace('AV', 'AVENUE', regex=True)
speedzones['ROAD_NAME'] = speedzones["ROAD_NAME"].replace('CCT', 'CIRCUIT', regex=True)
speedzones['ROAD_NAME'] = speedzones["ROAD_NAME"].replace('CL', 'CLOSE', regex=True)
speedzones['ROAD_NAME'] = speedzones["ROAD_NAME"].replace('VW', 'VIEW', regex=True)
speedzones['ROAD_NAME'] = speedzones["ROAD_NAME"].replace('WY', 'WAY', regex=True)
speedzones['ROAD_NAME'] = speedzones["ROAD_NAME"].replace('PL', 'PLACE', regex=True)

<h3>6. Define a Distance Function</h3>
<p>Defnes a <code>distance</code> function to calculate the great-circle distance between two points on the Earth, given their latitudes and longitudes, using the Haversine formula.</p>


In [6]:
def distance(s_lat, s_lng, e_lat, e_lng):

   # Approximate radius of earth in km
   R = 6373.0

   s_lat = s_lat*np.pi/180.0
   s_lng = np.deg2rad(s_lng)
   e_lat = np.deg2rad(e_lat)
   e_lng = np.deg2rad(e_lng)

   d = np.sin((e_lat - s_lat)/2)**2 + np.cos(s_lat)*np.cos(e_lat) * np.sin((e_lng - s_lng)/2)**2

   return 2 * R * np.arcsin(np.sqrt(d))




<h3>7. Initialize a Folium Map Object</h3>
<p>Initializs a <code>folium</code> map object centered around a specific latitude and longitud, in this case the ACT Australias.</p>

In [7]:
map_osm = folium.Map(location=[-35.23590198092005, 149.14658730303773])
#map_osm

<h3>8. Iterate Through <code>speedcameralocations</code></h3>
<p>Iterate through <code>speedcameralocations</code> to find matching speed zones based on street names. Then calculate the distances from each camera to the start and end points of these speed zones. If these distances are within 50 meters, mark the locations on the map with <code>folium.CircleMarker</code>.</p>
<p>The final result is a <code>folium</code> map (<code>map_osm</code>) with marked locations illustrating the proximity of traffic cameras to the starts and ends of speed zones.</p>

In [8]:

    

# Convert 'ROAD_NAME' column to lowercase for case-insensitive matching
speedzones['ROAD_NAME'] = speedzones['ROAD_NAME'].str.lower()

# Iterate through speedcameralocations and find matches
closezonend = []
closezonest = []
matches = []

for index, sepeedcameralocation in speedcameralocations.iterrows():
    street_name = sepeedcameralocation["Street"].lower()
    
    #find all speedzones on the road
    match = speedzones[speedzones['ROAD_NAME'].str.contains(street_name, na=False, case=False)]
    
    #set a large value for distance to start the loop
    olddistancest = 999999999
    olddistancend = 999999999
    
    #find the closest start and end of speed zones to the cameras
    for index, m in match.iterrows():

        newdistancest = (distance(sepeedcameralocation["LATITUDE"],sepeedcameralocation["LONGITUDE"],m['startlat'],m['startlong']))
        
        newdistancend = (distance(sepeedcameralocation["LATITUDE"],sepeedcameralocation["LONGITUDE"],m['endlat'],m['endlong']))
        
        if (newdistancest < olddistancest): 
            olddistancest = newdistancest
            stzone = m
        else:
            olddistancest = olddistancest
            
        if (newdistancend < olddistancend): 
            olddistancend = newdistancend
            ndzone = m
        else:
            olddistancend = olddistancend        
            
            
    
    #find speed zone starts that are within 50m of a camera location and mark them on the map
    if (olddistancest < 0.05):
        folium.CircleMarker(location=[stzone["startlat"], stzone["startlong"]],radius = 5).add_to(map_osm)
        folium.CircleMarker(location=[sepeedcameralocation["LATITUDE"],sepeedcameralocation["LONGITUDE"]],radius = 5,color = 'red', text = 'test').add_to(map_osm)
        
        
        closezonest.append(stzone)    
    else:
        None
        
    if (olddistancend < 0.05):
        folium.CircleMarker(location=[ndzone["startlat"], ndzone["startlong"]],radius = 5,color = 'orange').add_to(map_osm)
        
        
        closezonend.append(ndzone)    
    else:
        None
    


<ipython-input-8-2ffdfa206f73>:13: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  match = speedzones[speedzones['ROAD_NAME'].str.contains(street_name, na=False, case=False)]


In [9]:
#result

map_osm  